In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import root_mean_squared_error

In [4]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/746342380452218530', creation_time=1754040054151, experiment_id='746342380452218530', last_update_time=1754040054151, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:

def read_dataframe(filename):
    df = pd.read_parquet(filename, engine="pyarrow")
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    return df

In [6]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

In [7]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()
train_dicts = df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df[target].values
y_val = df[target].values

In [9]:
import xgboost as xgb

In [10]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [11]:
with mlflow.start_run():
    params = {
        'learning_rate': 0.06431075452279457,
        'max_depth': 45,
        'min_child_weight': 3.9809358849019483,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.021831790170560292,
        'reg_lambda': 0.008839071537844754,
        'seed': 42
    }

    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round = 1000,
        evals=[(valid,'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    import os
    os.makedirs("models", exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models")

[0]	validation-rmse:11.60165
[1]	validation-rmse:11.07778
[2]	validation-rmse:10.59717
[3]	validation-rmse:10.15795
[4]	validation-rmse:9.75647
[5]	validation-rmse:9.38970
[6]	validation-rmse:9.05618
[7]	validation-rmse:8.75350
[8]	validation-rmse:8.47681
[9]	validation-rmse:8.22735
[10]	validation-rmse:8.00160
[11]	validation-rmse:7.79673
[12]	validation-rmse:7.61256
[13]	validation-rmse:7.44748
[14]	validation-rmse:7.29835
[15]	validation-rmse:7.16387
[16]	validation-rmse:7.04436
[17]	validation-rmse:6.93621
[18]	validation-rmse:6.84021
[19]	validation-rmse:6.75195
[20]	validation-rmse:6.67288
[21]	validation-rmse:6.60332
[22]	validation-rmse:6.54069
[23]	validation-rmse:6.48354
[24]	validation-rmse:6.43263
[25]	validation-rmse:6.38714
[26]	validation-rmse:6.34547
[27]	validation-rmse:6.30829
[28]	validation-rmse:6.27507
[29]	validation-rmse:6.24481
[30]	validation-rmse:6.21772
[31]	validation-rmse:6.19296
[32]	validation-rmse:6.17081
[33]	validation-rmse:6.15003
[34]	validation-rmse

2025/08/01 09:28:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/ubuntu/anaconda3/envs/mlops-training/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [09:28:36] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/08/01 09:28:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ambitious-goat-463 at: http://localhost:5000/#/experiments/746342380452218530/runs/bd5ab112f78e433aaeab835391723c70
🧪 View experiment at: http://localhost:5000/#/experiments/746342380452218530
